# Obteniendo el número de casos positivos, fallecidos y vacunados contra COVID-19 por semana epidemiológica

En este notebook se busca obtener un dataset con el número de **casos positivos**, **fallecidos** y **totalmente vacunados** tanto a **nivel nacional**, por **grupo etáreo** como por **departamento** de todo el Perú.


## A. Lectura y Procesamiento

En esta sección se leen y preprocesan los dataset de **RawData**. 

El dataset de casos positivos **(positivos_covid.csv)** y vacunados **(TB_VACUNACION_COVID19.csv)** no están incluidos en el repositorio debido a su excesivo tamaño ~170 MB y ~2.3 GB respectivamente. Sin embargo se puede descargar directamente desde la página de Datos Abiertos de COVID-19 de MINSA y colocarlos en la dirección RawData:
- [Casos positivos por COVID-19](https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa)
- [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion) 

Para preservar recursos, no se procesará la información usando **Pandas** en estos datasets. En su lugar se utilizará el paquete **Dask**. El dataset no será un **pandas dataframe** si no un **dask dataframe** el cual está separado en *n* particiones de tipo **pandas dataframe**.

### Cargar Librerías

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from epiweeks import Week

import functions as fn

### Dataset de casos positivos

In [2]:
# Leer el dataset (dask dataframe ddf)
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO']
ddf_cas = dd.read_csv(url_cas, sep = ';', usecols = col_cas, 
                      dtype={'EDAD': 'float64', 'FECHA_RESULTADO': 'float64'})
ddf_cas = ddf_cas.dropna().astype({'EDAD': 'int8', 'FECHA_RESULTADO': 'int32'})

# Convertir la columna "fecha_vacunacion" a formato fecha y obtenener el año y semana epidemiológica
ddf_cas = ddf_cas.assign(fecha_vacunacion = dd.to_datetime(ddf_cas['FECHA_RESULTADO'], 
                                                           format = "%Y%m%d", errors="coerce"))
ddf_cas['fecha_vacunacion'] = ddf_cas['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
ddf_cas[['year','epi_week']] = ddf_cas['fecha_vacunacion'].str.split("W", 1, expand=True)
ddf_cas = ddf_cas.astype({'year': 'int16', 'epi_week': 'int8'})

# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')

ddf_cas['edad_cat'] = ddf_cas.map_partitions(edad_cat, 'EDAD')

# Drop unused cols
ddf_cas = ddf_cas.drop(['FECHA_RESULTADO', 'fecha_vacunacion', 'EDAD'], axis=1)

### Dataset de fallecidos

In [3]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'DEPARTAMENTO', 'EDAD_DECLARADA']
df_fal = pd.read_csv(url_fal, sep = ';', usecols = col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32', 
                                                                     'EDAD_DECLARADA': 'int8'})
# Transformar a formato fecha y convertir a semana epidemiológia y año
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')
df_fal[['year','epi_week']] = df_fal['FECHA_FALLECIMIENTO'].apply(lambda date_name : Week.fromdate(date_name) \
                                                                                         .weektuple()).tolist()
# Categorizar la edad en grupos etáreos
df_fal['edad_cat'] = edad_cat(df_fal, 'EDAD_DECLARADA')

#df_fal['SEXO'].replace({'MASCULINO': 0,'FEMENINO':1}, inplace=True)
#df_fal = df_fal[df_fal['SEXO'].apply(lambda x: str(x).isdigit())]

# Change types and drop unused cols
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8'}).drop(['FECHA_FALLECIMIENTO', 'EDAD_DECLARADA'], axis=1)

### Dataset de vacunados

Para encontrar el departamento respectivo de cada vacunado se prevee lo siguiente:

1) El dataset de vacunados **(TB_VACUNACION_COVID19.csv)** solamente proporciona información sobre la ubicación del establecimientos de salud (eess) llamado **'id_eess'**.

2) El dataset de los centros de vacunación [(TB_CENTRO_VACUNACION.csv)](/RawData/TB_CENTRO_VACUNACION.csv) puede ser utilizado para hacer *match* del **'id_eess'** del dataset de vacunados con la variable **'id_ubigeo'**. Que es una variable numérica que representa a cada lugar.

3) Finalmente, con el dataset de ubigeo [(TB_UBIGEOS.csv)](/RawData/TB_UBIGEOS.csv) es posible reemplazar a cada **'id_ubigeo'** con la variable **'departamento'** respectiva.

Para el dataset de UBIGEO solo se necesita las columnas **'id_ubigeo'** y **'departamento'**. Para el dataset de CENTROS DE VACUNACIÓN solo se necesitan las columnas **'id_centro de vacunación'** e **'id_ubigeo'**.

In [4]:
# Leer los datasets de ubigeo y centros de vacunación
url_ubigeo = '../RawData/TB_UBIGEOS.csv'
url_vaccenter = '../RawData/TB_EESS.csv'
ubigeo = pd.read_csv(url_ubigeo, usecols = ['id_ubigeo', 'departamento'])
vaccenter = pd.read_csv(url_vaccenter, usecols= ['id_eess','id_ubigeo'])

# Unimos ambos dataset mediante 'id_ubigeo'
vaccenter = vaccenter.merge(ubigeo, on = 'id_ubigeo', how = 'left')
del ubigeo, vaccenter['id_ubigeo']

In [5]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['fecha_vacunacion','dosis', 'edad', 'id_eess']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'edad': 'float64'})
# Personas con 3 o menos dosis
ddf_vac = ddf_vac[ddf_vac['dosis'] <= 3].dropna() 

# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce"))
ddf_vac['epi_date'] = ddf_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
ddf_vac[['year','epi_week']] = ddf_vac['epi_date'].str.split('W', 1, expand=True)
ddf_vac = ddf_vac.astype({'year': 'int16', 'epi_week': 'int8', 'edad': 'int8'})

# Categorizar la variable "edad" en grupos etáreos
ddf_vac['edad_cat'] = ddf_vac.map_partitions(edad_cat, 'edad')

# Encontrar la variable "departamento" usando "id_eess"
ddf_vac = ddf_vac.merge(vaccenter, how='left', on='id_eess')

# Drop unused cols
ddf_vac = ddf_vac.drop(['fecha_vacunacion', 'epi_date', 'edad', 'id_eess'], axis=1)

## B. Total por cada semana epidemiológica

En esta sección se busca obtener el número **total** de **casos positivos**, **fallecidos** y **vacunados (dosis 1,2 y 3)** por cada **año** y **semana epidemiológica** de todo el Perú. 

In [6]:
# Contar el número de CASOS por año y semana epidemiológica
epi_cas = ddf_cas.groupby(['year', 'epi_week']).count().rename(columns = {'edad_cat':'total_casos'}).compute()
del epi_cas['DEPARTAMENTO']

In [7]:
# Contar el número de FALLECIDOS por año y semana epidemiológica
epi_fal = df_fal.groupby(['year', 'epi_week']).count()
epi_fal = epi_fal.rename(columns = {'DEPARTAMENTO':'total_fallecidos'})
del epi_fal['edad_cat']

In [8]:
# Contar el número de VACUNADOS (dosis 1, dosis 2 y dosis 3) por año y semana epidemiológica
""" Nota: No se puede usar multi-index en la función pivot_table para dask, debido a eso se crea la
función 'crosstab4dask'. Aún es necesario buscar en dask funciones para iterar todas las particiones
más facilmente y optimizar el script. TAREA PENDIENTE """

def crosstab4dask(ddf, var_name):
    """ Función que recibe un dask dataframe 'ddf' y una variable de entrada 'var_name' y realiza 
    una crosstab multi-index de dicha variable por 'year' y 'epi_week' """
    
    lst = []  # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        df = ddf.partitions[i].compute()
        lst.append(pd.crosstab(index=[df['year'],df['epi_week']], columns = df[var_name]))

    merged_epivac = pd.concat(lst, axis=1).fillna(0).astype(np.int64).groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [9]:
epi_dose = crosstab4dask(ddf_vac, 'dosis')

In [10]:
# Unir los dataframes de casos positivos, fallecidos y dosis de vacunación por año y semana epidemiológica
epiweeks = pd.concat([epi_cas, epi_fal, epi_dose], axis=1) \
             .fillna(0).astype('int64')

epiweeks.rename(columns={1: 'total_dosis_1', 
                         2: 'total_dosis_2', 
                         3: 'total_dosis_3'}, 
                inplace=True)
                              
epiweeks

total_casos  total_fallecidos  total_dosis_1  total_dosis_2  \
year epi_week                                                                
2020 10                  6                 3              0              0   
     11                 51                 3              0              0   
     12                279                36              0              0   
     13                458                64              0              0   
     14               1993               227              0              0   
...                    ...               ...            ...            ...   
2021 50              10611               277         640927         855115   
     51              10889               243         233117         402492   
     52              22936               274         157071         343609   
2022 1               92597               292         162998         445020   
     2              163737               299         101312         253042   

               total_dosis_3  
year epi_week                 
2020 10                    0  
     11                    0  
     12                    0  
     13                    0  
     14                    0  
...                      ...  
2021 50               394184  
     51               832979  
     52               878193  
2022 1               1208075  
     2                754601  

[98 rows x 5 columns]

## C. Por grupo etáreo de todo el Perú

En esta sección se busca obtener el número de **casos positivos**, **fallecidos** y **personas completamente vacunadas** contra COVID-19 por **grupo etáreo**. Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

In [11]:
# Contar el número de CASOS por grupo etáreo por año y semana epidemiológica
age_dic = {0: '0_17_cas', 1:'18_29_cas',  2: '30_39_cas', 3:'40_49_cas',
           4: '50_59_cas', 5:'60_69_cas', 6:'70_79_cas', 7:'80_more_cas'}  # Age dictionary

df_cas_edad = crosstab4dask(ddf_cas, 'edad_cat').rename(age_dic, axis=1)

In [12]:
# Contar el número de FALLECIDOS por grupo etáreo por año y semana epidemiológica
age_dic = {0: '0_17_fal', 1:'18_29_fal',  2: '30_39_fal', 3:'40_49_fal',
           4: '50_59_fal', 5:'60_69_fal', 6:'70_79_fal', 7:'80_more_fal'}  # Age dictionary

df_fal_edad = pd.crosstab(index = [df_fal['year'], df_fal['epi_week']], 
                          columns = df_fal['edad_cat']).rename(age_dic, axis=1)

In [13]:
# Contar el número de TOTALMENTE VACUNADOS (2 dosis) por grupo etáreo por año y semana epidemiológica
age_dic = {0: '0_17_vac', 1:'18_29_vac',  2: '30_39_vac', 3:'40_49_vac',
           4: '50_59_vac', 5:'60_69_vac', 6:'70_79_vac', 7:'80_more_vac'}  # Age dictionary

df_vac_edad = crosstab4dask(ddf_vac, 'edad_cat').rename(age_dic, axis=1)

In [14]:
# Unir los dataframes de grupos etáreos por año y semana epidemiológica
epiweeks_edad = pd.concat([df_cas_edad, df_fal_edad, df_vac_edad], axis=1) \
                  .fillna(0).astype('int64')

epiweeks_edad.rename(columns={1: 'total_dosis_1', 2: 'total_dosis_2', 3: 'total_dosis_3'}, inplace=True)
                             
epiweeks_edad

edad_cat       0_17_cas  18_29_cas  30_39_cas  40_49_cas  50_59_cas  \
year epi_week                                                         
2020 10               1          3          0          0          1   
     11               5         16          8         11          2   
     12              16         69         56         49         38   
     13              12         59        107         95         82   
     14              69        424        429        356        362   
...                 ...        ...        ...        ...        ...   
2021 50             698       2730       2738       1811       1266   
     51             846       2894       2908       1852       1271   
     52            1899       6281       5921       3990       2649   
2022 1             6102      31125      24209      14795       8945   
     2            11023      48633      43218      28962      17361   

edad_cat       60_69_cas  70_79_cas  80_more_cas  0_17_fal  18_29_fal  ...  \
year epi_week                                                          ...   
2020 10                0          1            0         0          1  ...   
     11                6          3            0         2          0  ...   
     12               34         12            5         0          2  ...   
     13               65         27           11         5          3  ...   
     14              206         97           50         3          5  ...   
...                  ...        ...          ...       ...        ...  ...   
2021 50              810        346          212         2          7  ...   
     51              631        293          194         2          4  ...   
     52             1342        570          284         5         10  ...   
2022 1              4576       1933          912         3          6  ...   
     2              9094       3689         1757         7          3  ...   

edad_cat       70_79_fal  80_more_fal  0_17_vac  18_29_vac  30_39_vac  \
year epi_week                                                           
2020 10                0            1         0          0          0   
     11                1            0         0          0          0   
     12                8            6         0          0          0   
     13               16           10         0          0          0   
     14               52           48         0          0          0   
...                  ...          ...       ...        ...        ...   
2021 50               64          102    352178     573222     295715   
     51               63           82    158955     250804     193434   
     52               64           82    115788     209234     244335   
2022 1                62           93    125110     342470     383495   
     2                55          101     73286     247853     242097   

edad_cat       40_49_vac  50_59_vac  60_69_vac  70_79_vac  80_more_vac  
year epi_week                                                           
2020 10                0          0          0          0            0  
     11                0          0          0          0            0  
     12                0          0          0          0            0  
     13                0          0          0          0            0  
     14                0          0          0          0            0  
...                  ...        ...        ...        ...          ...  
2021 50           165930     123869     211228     117346        50738  
     51           282046     326604     155339      70311        31095  
     52           305545     305767     124551      50617        23036  
2022 1            399636     349106     139392      53735        23149  
     2            226997     191462      80775      32126        14359  

[98 rows x 24 columns]

## D. Por cada departamento del Perú

En esta sección se busca obtener el número de **casos positivos**, **fallecidos** y **personas completamente vacunadas** contra COVID-19 para cada **departamento**. Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

In [15]:
# Contar el número de CASOS por departamento respecto al año y semana epidemiológica
df_cas_dep = crosstab4dask(ddf_cas, 'DEPARTAMENTO').add_suffix('_cas')

In [16]:
# Contar el número de FALLECIDOS por departamento respecto al año y semana epidemiológica
df_fal_dep = pd.crosstab(index=[df_fal['year'], df_fal['epi_week']],
                         columns=[df_fal['DEPARTAMENTO']],
                         margins = False).add_suffix('_fal')

In [17]:
# Contar el número de personas TOTALMENTE VACUNADAS por departamento respecto al año y semana epidemiológica
df_vac_dep = crosstab4dask(ddf_vac, 'departamento').add_suffix('_vac')

In [18]:
epiweeks_dep = pd.concat([df_cas_dep, df_fal_dep, df_vac_dep], axis = 1).fillna(0)

epiweeks_dep

AMAZONAS_cas  ANCASH_cas  APURIMAC_cas  AREQUIPA_cas  \
year epi_week                                                         
2020 10                   0           0             0             1   
     11                   0           0             0             1   
     12                   0           4             0             2   
     13                   0           2             0            11   
     14                   0          18             1            36   
...                     ...         ...           ...           ...   
2021 50                 165         403            50           199   
     51                 170         468            34           178   
     52                 258         555            45           639   
2022 1                  485        2725           315          4252   
     2                  773        5774           947          9756   

               AYACUCHO_cas  CAJAMARCA_cas  CALLAO_cas  CUSCO_cas  \
year epi_week                                                       
2020 10                   0              0           0          0   
     11                   0              0           1          1   
     12                   0              0           5          3   
     13                   1              1          17         13   
     14                   2              4         239         43   
...                     ...            ...         ...        ...   
2021 50                  97            217         339        144   
     51                  63            207         342        121   
     52                  97            376         909        513   
2022 1                  551           1883        4299       4992   
     2                 1696           3592        6141       7980   

               HUANCAVELICA_cas  HUANUCO_cas  ...  LORETO_vac  \
year epi_week                                 ...               
2020 10                       0            0  ...         0.0   
     11                       0            2  ...         0.0   
     12                       0            0  ...         0.0   
     13                       0            2  ...         0.0   
     14                       2            2  ...         0.0   
...                         ...          ...  ...         ...   
2021 50                      66          223  ...     67553.0   
     51                      59          116  ...     34043.0   
     52                      41          211  ...     32538.0   
2022 1                      212          768  ...     33159.0   
     2                      707         1885  ...     30929.0   

               MADRE DE DIOS_vac  MOQUEGUA_vac  PASCO_vac  PIURA_vac  \
year epi_week                                                          
2020 10                      0.0           0.0        0.0        0.0   
     11                      0.0           0.0        0.0        0.0   
     12                      0.0           0.0        0.0        0.0   
     13                      0.0           0.0        0.0        0.0   
     14                      0.0           0.0        0.0        0.0   
...                          ...           ...        ...        ...   
2021 50                  19102.0        9041.0     9958.0   118609.0   
     51                   7179.0        8821.0    11912.0    71622.0   
     52                   4620.0        5585.0    14470.0    67570.0   
2022 1                    9187.0        8565.0    16699.0    89113.0   
     2                    4860.0        7510.0    12243.0    54758.0   

               PUNO_vac  SAN MARTIN_vac  TACNA_vac  TUMBES_vac  UCAYALI_vac  
year epi_week                                                                
2020 10             0.0             0.0        0.0         0.0          0.0  
     11             0.0             0.0        0.0         0.0          0.0  
     12             0.0             0.0        0.0         0.0          0.0  
     13             

## E. Dataset Final

En esta sección se unen todos los dataframes obtenidos de **casos positivos**, **fallecidos** y **personas completamente vacunadas** respecto a grupos etáreos, y departamentos por **año** y **semana epidemiológica**. El dataset obtenido *(Data Product 1)* puede encontrarse en el directorio **Data** o clic en [DP1_covid19-peru_x_semanaEpi.csv](Data/DP1_covid19-peru_x_semanaEpi.csv)

In [19]:
epiweeks_final = pd.concat([epiweeks, epiweeks_edad, epiweeks_dep ], axis = 1).fillna(0).astype('int64')
epiweeks_final

total_casos  total_fallecidos  total_dosis_1  total_dosis_2  \
year epi_week                                                                
2020 10                  6                 3              0              0   
     11                 51                 3              0              0   
     12                279                36              0              0   
     13                458                64              0              0   
     14               1993               227              0              0   
...                    ...               ...            ...            ...   
2021 50              10611               277         640927         855115   
     51              10889               243         233117         402492   
     52              22936               274         157071         343609   
2022 1               92597               292         162998         445020   
     2              163737               299         101312         253042   

               total_dosis_3  0_17_cas  18_29_cas  30_39_cas  40_49_cas  \
year epi_week                                                             
2020 10                    0         1          3          0          0   
     11                    0         5         16          8         11   
     12                    0        16         69         56         49   
     13                    0        12         59        107         95   
     14                    0        69        424        429        356   
...                      ...       ...        ...        ...        ...   
2021 50               394184       698       2730       2738       1811   
     51               832979       846       2894       2908       1852   
     52               878193      1899       6281       5921       3990   
2022 1               1208075      6102      31125      24209      14795   
     2                754601     11023      48633      43218      28962   

               50_59_cas  ...  LORETO_vac  MADRE DE DIOS_vac  MOQUEGUA_vac  \
year epi_week             ...                                                
2020 10                1  ...           0                  0             0   
     11                2  ...           0                  0             0   
     12               38  ...           0                  0             0   
     13               82  ...           0                  0             0   
     14              362  ...           0                  0             0   
...                  ...  ...         ...                ...           ...   
2021 50             1266  ...       67553              19102          9041   
     51             1271  ...       34043               7179          8821   
     52             2649  ...       32538               4620          5585   
2022 1              8945  ...       33159               9187          8565   
     2             17361  ...       30929               4860          7510   

               PASCO_vac  PIURA_vac  PUNO_vac  SAN MARTIN_vac  TACNA_vac  \
year epi_week                                                              
2020 10                0          0         0               0          0   
     11                0          0         0               0          0   
     12                0          0         0               0          0   
     13                0          0         0               0          0   
     14                0          0         0               0          0   
...                  ...        ...       ...             ...        ...   
2021 50             9958     118609     93243           48510      14664   
     51            11912      71622     52147           27299      12821   
     52            14470      67570     41284           20417      10991   
2022 1             16699      89113     66477           36045      17803   
     2             12243      54758     47366           29788      10827   

               TUMBE

In [20]:
epiweeks_final.to_csv('../Data/DP1_covid19-peru_x_semanaEpi.csv')